## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rawson,AR,-43.3002,-65.1023,54.21,74,9,19.17,clear sky
1,1,Thompson,CA,55.7435,-97.8558,14.16,92,90,5.75,overcast clouds
2,2,Komsomolskiy,UZ,40.4272,71.7189,42.22,52,2,4.61,haze
3,3,West Wendover,US,40.7391,-114.0733,34.81,44,90,0.00,overcast clouds
4,4,Zemio,CF,5.0314,25.1361,81.46,62,87,3.83,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
mintemp=float(input("What is the lowest temperature you would like?"))
maxtemp=float(input("What is the highest temperature you would like?"))



What is the lowest temperature you would like?45
What is the highest temperature you would like?65


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
eligible_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= maxtemp) & (city_data_df["Max Temp"] >= mintemp)]
eligible_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rawson,AR,-43.3002,-65.1023,54.21,74,9,19.17,clear sky
5,5,Mataura,NZ,-46.1927,168.8643,57.31,54,95,7.31,overcast clouds
7,7,Skibbereen,IE,51.5500,-9.2667,48.00,87,93,8.48,overcast clouds
9,9,Vincennes,FR,48.8500,2.4333,48.00,94,90,4.61,overcast clouds
11,11,Ardakan,IR,32.3100,54.0175,52.18,32,0,1.30,clear sky
18,18,Fortuna,US,40.5982,-124.1573,57.09,70,90,1.01,overcast clouds
21,21,Nouadhibou,MR,20.9310,-17.0347,64.38,94,12,9.22,few clouds
23,23,Lebu,CL,-37.6167,-73.6500,51.94,94,43,16.24,scattered clouds
31,31,Constitucion,CL,-35.3333,-72.4167,55.54,74,100,2.64,overcast clouds
34,34,Bethel,US,41.3712,-73.4140,45.48,88,100,1.01,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
eligible_cities_df.isnull().sum()


City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_eligible_cities=eligible_cities_df.dropna()
clean_eligible_cities.count()


City_ID                162
City                   162
Country                162
Lat                    162
Lng                    162
Max Temp               162
Humidity               162
Cloudiness             162
Wind Speed             162
Current Description    162
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_eligible_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rawson,AR,54.21,clear sky,-43.3002,-65.1023,
5,Mataura,NZ,57.31,overcast clouds,-46.1927,168.8643,
7,Skibbereen,IE,48.00,overcast clouds,51.5500,-9.2667,
9,Vincennes,FR,48.00,overcast clouds,48.8500,2.4333,
11,Ardakan,IR,52.18,clear sky,32.3100,54.0175,
18,Fortuna,US,57.09,overcast clouds,40.5982,-124.1573,
21,Nouadhibou,MR,64.38,few clouds,20.9310,-17.0347,
23,Lebu,CL,51.94,scattered clouds,-37.6167,-73.6500,
31,Constitucion,CL,55.54,overcast clouds,-35.3333,-72.4167,
34,Bethel,US,45.48,overcast clouds,41.3712,-73.4140,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
    

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [36]:
import numpy as np

In [39]:
# 7. Drop the rows where there is no Hotel Name.

hotel_df = hotel_df.replace(r'^\s*$', np.NaN, regex=True)
clean_hotel_df=hotel_df.dropna()

City                   162
Country                162
Max Temp               162
Current Description    162
Lat                    162
Lng                    162
Hotel Name             157
dtype: int64

In [40]:
clean_hotel_df=hotel_df.dropna()
clean_hotel_df.count()

City                   157
Country                157
Max Temp               157
Current Description    157
Lat                    157
Lng                    157
Hotel Name             157
dtype: int64

In [41]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [44]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Max Temp} °F and {Current Description}</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [46]:
# 11a. Add a marker layer for each city to the map. 

locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))